#Solving Combinatorial optimization problem using PSO


> PSO for combinatorial optimization problem


In [41]:
#the utilities
import math
import random
import matplotlib.pyplot as plt

class City:
  def __init__(self, x, y):
    self.x = x
    self.y = y

  #def distance(self, city, route):
    #return abs(self.x - city.x)
    #return math.hypot(self.x - city.x, self.y - city.y)

  def __repr__(self):
    #return f"({self.x})
    return f"({self.x}, {self.y})"


def read_cities(size):
  cities = []
  with open(f'test_data/cities_{size}.data', 'r') as handle:
      lines = handle.readlines()
      for line in lines:
          x, y = map(float, line.split())
          cities.append(City(x, y))
  return cities


def write_cities_and_return_them(size):
  cities = generate_cities(size)
  with open(f'test_data/cities_{size}.data', 'w+') as handle:
      for city in cities:
          handle.write(f'{city.x} {city.y}\n')
  return cities

#can add size and customize it further too
def generate_cities():
  return[City(x=0,y='hole-1'), City(x=1,y='hole-2'), City(x=2,y='hole-3'), City(x=3,y='hole-4'), City(x=4,y='hole-5'), City(x=5,y='hole-6')]


#defining the cost matrix
def cost_matrix():
  return [[0,1.3,0.5,2.6,4.1,3.2],
          [1.3,0,3.5,4.7,3.0,5.3],
          [0.5,3.5,0,3.5,4.6,6.2],
          [2.6,4.7,3.5,0,3.8,0,1.9],
          [4.1,3.0,4.6,3.8,0,1.9],
          [3.2,5.3,6.2,0.9,1.9,0]]

#calculate the path cost using the cost matrix
def path_cost(route):
  cMatrix = cost_matrix()
  psum = sum([cMatrix[city.x][route[index-1].x] for index, city in enumerate(route)])
  return psum 
  


def visualize_tsp(title, cities):
  fig = plt.figure()
  fig.suptitle(title)
  x_list, y_list = [], []
  for city in cities:
      x_list.append(city.x)
      y_list.append(city.y)
  x_list.append(cities[0].x)
  y_list.append(cities[0].y)

  plt.plot(x_list, y_list, 'ro')
  plt.plot(x_list, y_list, 'g')
  plt.show(block=True)

>Implementation


In [42]:
import random
import math
import matplotlib.pyplot as plt

#the particle class
class Particle:
  def __init__(self,route,cost=None):
    self.route = route
    self.pbest = route
    self.current_cost = cost if cost else self.path_cost()
    self.pbest_cost = cost if cost else self.path_cost()
    self.velocity = []

  def clearVelocity(self):
    self.velocity.clear()

  #updating cost and personal best value  
  def updateCostandPbest(self):
    self.current_cost = self.path_cost()
    if self.current_cost < self.pbest_cost:
      self.pbest = self.route
      self.pbest_cost = self.current_cost
  #returns path cost
  def path_cost(self):
    return path_cost(self.route)
  
class PSO:
  
  def __init__(self, iterations, population_size, gbest_prob = 0.02, pbest_prob = 0.9, cities = None):
    self.cities = cities
    self.gbest = None
    self.gcost_iter = []
    self.iterations = iterations
    self.population_size = population_size
    self.particles = []
    self.gbest_prob = gbest_prob
    self.pbest_prob = pbest_prob
    
    #generating the swarm
    solutions = self.initial_population()
    self.particles = [Particle(route=solution) for solution in solutions]

  #generating random routes for particles
  def random_route(self):
    return random.sample(self.cities,len(self.cities))
  
  #generating a greedy route for one particle of the initial population
  def greedy_route(self,start_index):
    unvisited = self.cities[:]
    del unvisited[start_index]
    
    route = [self.cities[start_index]]
    cMatrix = cost_matrix()

    while len(unvisited):
      index, nearest_city = min(enumerate(unvisited), key=lambda item: cMatrix[item[1].x][route[-1].x])
      route.append(nearest_city)
      del unvisited[index]
    return route

  #generating an intial population for testing 
  #assuming n as the population size then we randomly generate n-1 sequences
  #and one greedy sequence (certain to have good fitness) so as to give good 
  #convergence
  #return the initial generated population
  def initial_population(self):
    random_population = [self.random_route() for _ in range(self.population_size - 1)]
    greedy_population = [self.greedy_route(0)]
    return [*random_population, *greedy_population]

  #running the algorithm
  def run(self):
    self.gbest = min(self.particles, key = lambda p: p.pbest_cost)
    
    for t in range(self.iterations):
      self.gbest = min(self.particles, key=lambda p: p.pbest_cost)
      
      self.gcost_iter.append(self.gbest.pbest_cost)

      #velocity and position updation
      #as in continous PSO
      for particle in self.particles: 
        particle.clearVelocity()
        temp_velocity=[]
        gbest = self.gbest.pbest[:]
        new_route = particle.route[:]
        
        #creating the swap sequence for veloctiy
        for i in range(len(self.cities)):
          if new_route[i] != particle.pbest[i]:
            swap = (i, particle.pbest.index(new_route[i]), self.pbest_prob)
            temp_velocity.append(swap)
            new_route[swap[0]], new_route[swap[1]] = new_route[swap[1]], new_route[swap[0]]

        for i in range(len(self.cities)):
          if new_route[i] != gbest[i]:
            swap = (i, gbest.index(new_route[i]), self.gbest_prob)
            temp_velocity.append(swap)
            gbest[swap[0]], gbest[swap[1]] = gbest[swap[1]], gbest[swap[0]]

        particle.velocity = temp_velocity

        #applying the swap sequence to the velocities w.r.t the prob value
        for swap in temp_velocity:
          #if random.random() <= swap[2]:
          if random.uniform(0.0,1.0) <= swap[2]:
            new_route[swap[0]], new_route[swap[1]] = new_route[swap[1]], new_route[swap[0]]

        particle.route = new_route
        particle.updateCostandPbest()

#main function
if __name__ == "__main__":
  cities = generate_cities()
  pso = PSO(iterations = 1500, population_size = 30, pbest_prob = 0.9, gbest_prob = 0.02, cities = cities)
  pso.run()
  print(f'cost:{pso.gbest.pbest_cost}\t \nglobal_best_particle: {pso.gbest.pbest}')

  x_list, y_list = [], []
  for city in pso.gbest.pbest:
    x_list.append(city.x)
    y_list.append(city.y)
  x_list.append(pso.gbest.pbest[0].x)
  y_list.append(pso.gbest.pbest[0].y)

  #print("city indices",x_list)
  tour = ""
  for i in y_list:
    tour += i + '  '
  print("optimum sequence for completion is: ")
  print(tour)

cost:10.2	 
global_best_particle: [(5, hole-6), (3, hole-4), (2, hole-3), (0, hole-1), (1, hole-2), (4, hole-5)]
optimum sequence for completion is: 
hole-6  hole-4  hole-3  hole-1  hole-2  hole-5  hole-6  
